### colab에 옮겨서 실행해보자~
#### 사유: pdf 저장이 편함, mediapipe 설치가 안됨 이슈...

In [1]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 47.4 MB/s eta 0:00:00


In [2]:
import cv2
import mediapipe as mp
import os
import csv
# numpy version 1.24.3

앗싸 mediapipe 구간 성공

### 발레와 요가 구분하기

In [25]:
# ballet
image_folder = '/content/drive/MyDrive/Colab Notebooks/인명학/data/ballet/'
csv_file = 'first_ballet.csv'

In [27]:
# yoga
image_folder = '/content/drive/MyDrive/Colab Notebooks/인명학/data/yoga'
csv_file = 'first_yoga.csv'

In [28]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)  # True == 이미지 모드, False == 비디오 모드
# 이미지로 진행할 예정이기 때문에 True 그대로


with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([
        'Image',
        'left wrist X', 'left wrist Y',
        'left ankle X', 'left ankle Y',
        'right wrist X', 'right wrist Y',
        'right ankle X', 'right ankle Y',
        'hip X', 'hip Y',
        'Label'
    ]) # 손목 발목 골반?


for img_name in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_name)
    image = cv2.imread(img_path)
    if image is None:
        continue

    height, width, _ = image.shape


    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = pose.process(image_rgb)


    if result.pose_landmarks:
        left_wrist = result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
        left_ankle = result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE]
        right_wrist = result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]
        right_ankle = result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE]
        left_hip = result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
        right_hip = result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]
        # 어깨, 손목 외 무릎 추가할 시:
        # left_KNEE = result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE]


        # 절대 좌표
        left_wrist_x = int(left_wrist.x * width)
        left_wrist_y = int(left_wrist.y * height)
        left_ankle_x = int(left_ankle.x * width)
        left_ankle_y = int(left_ankle.y * height)
        right_wrist_x = int(right_wrist.x * width)
        right_wrist_y = int(right_wrist.y * height)
        right_ankle_x = int(right_ankle.x * width)
        right_ankle_y = int(right_ankle.y * height)
        left_hip_x = int(left_hip.x * width)
        left_hip_y = int(left_hip.y * height)
        right_hip_x = int(right_hip.x * width)
        right_hip_y = int(right_hip.y * height)

        # CSV
        with open(csv_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([
                img_name,
                left_wrist_x, left_wrist_y,
                left_ankle_x, left_ankle_y,
                right_wrist_x, right_wrist_y,
                right_ankle_x, right_ankle_y,
                left_hip_x, left_hip_y,
                right_hip_x, right_hip_y,
                # 'ballet'
                'yoga'
            ])

### 머신러닝 분류기 사용

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

ballet_data = pd.read_csv('first_ballet.csv')
yoga_data = pd.read_csv('first_yoga.csv')

all_data = pd.concat([ballet_data, yoga_data], ignore_index=True)
shuffled_data = all_data.sample(frac=1, random_state=42).reset_index(drop=True)

X = shuffled_data.drop(columns=['Image', 'Label'])
y = shuffled_data['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.75
